<a href="https://colab.research.google.com/github/filipperogerio/Passaport-AI-Agent/blob/main/Passaport_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Já pensou em se mudar para outro país?
Esse agente de IA irá te ajudar a entender qual o lugar mais adequado para o seu perfil.

In [213]:
!pip install -q google-genai
!pip install -q google-adk
!pip install -q markdown pdfkit
!apt-get install -y wkhtmltopdf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wkhtmltopdf is already the newest version (0.12.6-2).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [214]:
#Inclusão de todos os imports necesseários para a execução do projeto
import warnings
import os
import requests # Para fazer requisições HTTP
import json # Para manipular os dados JSON
import textwrap # Para formatar melhor a saída de texto
import markdown
import pdfkit

from datetime import date
from IPython.display import display, Markdown , HTML# Para exibir texto formatado no Colab
from google.colab import files
from google.colab import userdata
from google import genai
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

warnings.filterwarnings("ignore")

In [181]:
#Configuração da variável de ambiente com a chave de API da Google

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [182]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [183]:
MODEL_ID = "gemini-2.0-flash"
client = genai.Client()

In [184]:
chat_config = types.GenerateContentConfig(
    system_instruction = """
      Você é consultor de imigração, será responsável por analisar o perfil das pessoas para determinar qual o melhor lugar
      para que elas possam migrar e viver um sonho. Você sempre vai responder de forma amigável e objetiva.
    """
)

chat = client.chats.create(model=MODEL_ID, config = chat_config)

In [185]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [208]:
################################################
# --- Agente 1: Agente de Perfil e Destino --- #
################################################

def agente_analista(perfil):
    analise_perfil = Agent(
        name="agente_analista_de_perfil_e_destino",
        model=MODEL_ID,
        description="Agente especializado em analisar o perfil do usuário e sugerir até três opções de destino para imigração.",
        tools=[google_search],
        instruction= f"""
        Você é um agente especializado em aconselhamento de imigração e realocação internacional.
        Sua principal tarefa é analisar detalhadamente o perfil fornecido pelo usuário: "{perfil}" e,
        utilizando sua vasta base de conhecimento sobre diversos países e cidades, combinada com a ferramenta de busca do Google (google_search),
        identificar e apresentar **três** opções de destino que melhor se adequem às suas características e preferências.

        Ao realizar a busca e seleção dos destinos, considere os seguintes aspectos do perfil do usuário:

        * Idade: (Considere como isso pode influenciar oportunidades de trabalho, estilo de vida e necessidades).
        * Interesses Culinários: (Quais tipos de culinária o usuário aprecia? Isso pode indicar países com forte tradição nessa área).
        * Área de Trabalho/Profissão: (Quais países têm mercados de trabalho aquecidos nessa área? Quais oferecem melhores oportunidades de carreira e salários?).
        * Idiomas: (Considere quais países falam os idiomas)
        * Gênero: (Embora não deva ser o fator principal, pode haver considerações culturais ou de segurança em alguns destinos).
        * Preferência por Estilo de Vida: (O usuário prefere lugares mais pacatos e com foco na natureza, ou cidades vibrantes com intensa vida social e cultural?).
        * Outras preferências ou necessidades específicas: (Se o usuário mencionar algo mais, como clima preferido, proximidade com familiares, etc., isso também deve ser levado em conta).
        * País onde Vive: (Desconsidere como destino o país que a pessoa está vivendo)
        **Restrições e Diretrizes:**

        * **Limite-se estritamente a apresentar apenas três opções de destino.**
        * Para cada opção de destino, forneça um breve resumo (1-2 frases) explicando **por que** essa opção foi considerada adequada
        ao perfil do usuário, destacando os aspectos que correspondem às suas preferências e necessidades.
        * Utilize a ferramenta `google_search` para obter informações atualizadas e relevantes sobre os países e cidades em relação aos
        critérios mencionados.
        * Priorize destinos que tenham uma combinação razoável de fatores que atendam ao perfil do usuário, mesmo que nenhum destino seja perfeito
         em todos os aspectos.
        * Se a informação fornecida pelo usuário for vaga ou insuficiente, utilize a busca do Google para explorar possibilidades amplas
        dentro das categorias mencionadas e faça inferências razoáveis, sempre indicando o raciocínio por trás da sugestão.

        **Formato de Saída Esperado:**

        Opção de Destino 1: [Nome do País/Cidade] - [Breve justificativa de adequação ao perfil].
        Opção de Destino 2: [Nome do País/Cidade] - [Breve justificativa de adequação ao perfil].
        Opção de Destino 3: [Nome do País/Cidade] - [Breve justificativa de adequação ao perfil].
        """
    )

    entrada_do_agente_analista = f"Perfil: {perfil}"
    analise = call_agent(analise_perfil, entrada_do_agente_analista)
    return analise


In [187]:
#####################################################
# --- Agente 2: Agente de Burocracia Migratória --- #
#####################################################

def agente_burocracia(destinos):
    burocracia_migratoria = Agent(
        name="agente_de_burocracia_migratoria",
        model=MODEL_ID,
        description="Agente especializado em pesquisar os requisitos de visto e documentação para diferentes países.",
        tools=[google_search],
        instruction= f"""
        Você é um agente especializado em informações sobre burocracia de imigração. Sua tarefa é pesquisar e fornecer detalhes essenciais sobre
        os requisitos de visto e documentação necessários para que um indivíduo possa imigrar para os seguintes destinos: "{destinos}".

        Para cada destino na lista, utilize a ferramenta de busca do Google (google_search) para encontrar informações precisas e atualizadas sobre:

        * **Tipos de Visto:** Quais são os tipos de visto disponíveis para diferentes propósitos (trabalho, estudo, residência permanente, etc.)?
        * **Requisitos de Elegibilidade:** Quais são os critérios que um indivíduo precisa atender para ser elegível para cada tipo de visto
        relevante?
        * **Documentação Necessária:** Quais documentos são exigidos para solicitar o visto (passaporte, certidões, comprovantes financeiros, etc.)?
        * **Processo de Solicitação:** Qual é o processo geral para solicitar o visto (formulários online, agendamentos, entrevistas, etc.)?
        * **Tempo Estimado de Processamento:** Qual é o tempo médio estimado para que a solicitação de visto seja processada?
        * **Links para Fontes Oficiais:** Forneça links para os sites oficiais de imigração ou consulados dos respectivos países para que o
        usuário possa obter informações detalhadas e atualizadas.

        **Restrições e Diretrizes:**

        * Forneça as informações de forma clara e organizada para cada destino.
        * Priorize informações de fontes oficiais (sites governamentais, consulados, embaixadas).
        * Seja conciso, mas forneça detalhes suficientes para que o usuário tenha uma compreensão geral dos requisitos.
        * Se houver diferentes tipos de visto relevantes para um imigrante em geral, mencione os principais.
        * Indique claramente os links para as fontes oficiais para cada tipo de informação (se possível).

        **Formato de Saída Esperado:**

        Para [Nome do País/Cidade 1]:
        - Tipos de Visto Relevantes: [Lista dos tipos de visto]
        - Requisitos de Elegibilidade (Geral): [Breve resumo dos principais requisitos]
        - Documentação Necessária (Geral): [Lista dos principais documentos]
        - Processo de Solicitação (Geral): [Breve descrição do processo]
        - Tempo Estimado de Processamento: [Tempo estimado]
        - Links Úteis: [Lista de links para sites oficiais]

        Para [Nome do País/Cidade 2]:
        - ... (seguir o mesmo formato)

        Para [Nome do País/Cidade 3]:
        - ... (seguir o mesmo formato)
        """
    )
    entrada_do_agente_burocracia = f"Destinos: {destinos}"
    burocracia = call_agent(burocracia_migratoria, entrada_do_agente_burocracia)
    return burocracia

In [188]:
#########################################################
# --- Agente 3: Agente de Análise de Custo de Vida e Moradia --- #
#########################################################

def agente_custo_vida_moradia(destinos):
    custo_vida_moradia = Agent(
        name="agente_analista_de_custo_de_vida_migratoria",
        model=MODEL_ID,
        description="Agente especializado em fornecer informações sobre custo de vida, salários, benefícios e opções de moradia em diferentes países.",
        tools=[google_search],
        instruction= f"""
        Você é um agente especializado em análise de custo de vida e informações sobre moradia internacional.
        Sua tarefa é pesquisar e fornecer detalhes relevantes sobre os custos de vida, salários, benefícios sociais e opções de moradia para os
        seguintes destinos: "{destinos}".

        Para cada destino na lista, utilize a ferramenta de busca do Google (google_search) para encontrar informações precisas e atualizadas sobre:

        * **Custo de Vida Mensal Estimado:** Detalhe os custos médios para itens como aluguel, alimentação, transporte, saúde, lazer e outros gastos básicos para uma pessoa ou uma família pequena.
        * **Salário Médio e Mínimo:** Informe o salário médio e o salário mínimo na área de trabalho relevante do usuário
        (se especificado anteriormente) ou para profissões em geral no país. Mencione a expectativa de renda para
        profissionais da área (se disponível).
        * **Benefícios Sociais:** Quais são os principais benefícios sociais oferecidos pelo governo
        (saúde pública, educação, auxílio-desemprego, etc.)?
        * **Opções de Moradia:** Descreva os tipos de moradia mais comuns
        (apartamentos, casas, albergues, etc.) e forneça uma faixa de preços médios para aluguel e compra em diferentes áreas (centrais, suburbanas). Se possível, utilize o Google Maps para identificar e mencionar brevemente áreas bem avaliadas para se morar, com base em critérios como segurança, infraestrutura e acesso a serviços.
        * **Impostos:** Mencione brevemente a estrutura geral de impostos sobre a renda.

        **Restrições e Diretrizes:**

        * Forneça as informações de forma clara e organizada para cada destino.
        * Tente fornecer valores em moeda local e, se possível, em uma moeda de referência comum (como USD ou EUR) para facilitar a comparação.
        * Seja conciso, mas forneça detalhes suficientes para que o usuário tenha uma boa compreensão da situação financeira e de moradia.
        * Utilize o Google Maps para auxiliar na identificação de áreas de moradia, mas não se limite a ele; inclua informações gerais
        sobre tipos de moradia e custos.
        * Mencione as fontes das informações encontradas (se possível, cite os sites consultados).

        **Formato de Saída Esperado:**

        Para [Nome do País/Cidade 1]:
        - Custo de Vida Mensal Estimado: [Valor em moeda local (e USD/EUR)]
            - Aluguel: [Valor]
            - Alimentação: [Valor]
            - Transporte: [Valor]
            - ...
        - Salário Médio/Mínimo (e Expectativa de Renda): [Valores em moeda local]
        - Benefícios Sociais: [Lista dos principais benefícios]
        - Opções de Moradia: [Descrição dos tipos e faixas de preço]
            - Áreas Bem Avaliadas (Google Maps): [Breve menção de áreas]
        - Impostos sobre a Renda: [Breve descrição]

        Para [Nome do País/Cidade 2]:
        - ... (seguir o mesmo formato)

        Para [Nome do País/Cidade 3]:
        - ... (seguir o mesmo formato)
        """
    )
    entrada_do_agente_custo_vida = f"Destinos: {destinos}"
    custo_vida = call_agent(custo_vida_moradia, entrada_do_agente_custo_vida)
    return custo_vida

In [189]:
#########################################################
# --- Agente 4: Agente de Insights Culturais e Vivenciais --- #
#########################################################

def agente_insights_culturais(destinos, perfil):
    insights_culturais = Agent(
        name="agente_de_insights_culturais_e_vivenciais",
        model=MODEL_ID,
        description="Agente especializado em encontrar vídeos do YouTube que ofereçam insights sobre a cultura e a vida em diferentes países, alinhados aos interesses do usuário.",
        tools=[google_search],
        instruction= f"""
            Você é um agente especializado em fornecer insights culturais e sobre a experiência de vida em diferentes países. Sua tarefa é encontrar vídeos relevantes do YouTube que ofereçam uma visão sobre como é morar nos seguintes destinos: "{destinos}", levando em consideração o perfil do usuário: "{perfil}".

            Para cada destino na lista, utilize a ferramenta de busca do Google (google_search) para encontrar vídeos do YouTube que abordem os seguintes aspectos (priorizando aqueles que se alinham aos interesses do perfil do usuário):

            * **Cultura e Costumes:** Vídeos que mostrem os costumes locais, tradições, etiqueta social e aspectos culturais importantes.
            * **Estilo de Vida:** Vídeos que ilustrem o dia a dia das pessoas, o ritmo de vida, as atividades de lazer comuns e a atmosfera
            geral do lugar.
            * **Comida e Bebida:** Vídeos sobre a culinária local, restaurantes, mercados e a cultura alimentar do país
            (especialmente relevante se o perfil do usuário mencionar interesses culinários).
            * **Comunidades de Expatriados (se aplicável):** Vídeos de outros imigrantes ou expatriados compartilhando suas experiências,
            desafios e dicas sobre morar no país.
            * **Atrações e Pontos Turísticos (sob a perspectiva de um morador):** Vídeos que mostrem os lugares interessantes para visitar,
            não apenas como turista, mas como alguém que vive ali.
            * **Segurança e Bem-Estar:** Vídeos que abordem questões de segurança e a qualidade de vida no país.

            **Restrições e Diretrizes:**

            * Encontre pelo menos 2-3 vídeos relevantes para cada destino.
            * **Forneça o título do vídeo e o link direto e completo para o vídeo no YouTube (exemplo: https://www.youtube.com/watch?v=VIDEO_ID).**
            * **Ao realizar a busca, priorize fortemente vídeos que sejam públicos e que permitam a incorporação (embedding).**
            * **Tente evitar resultados que pareçam ser vídeos privados, não listados, ou que em seus títulos ou descrições (visíveis nos resultados da busca) sugiram restrições de idade, restrições geográficas severas, ou que a incorporação esteja desabilitada.**
            * Inclua uma breve descrição (1-2 frases) de cada vídeo, destacando o que o usuário pode aprender sobre o país ao assisti-lo
            e como ele se relaciona com o perfil do usuário (se aplicável).
            * Priorize vídeos de pessoas que realmente moram ou moraram no país, em vez de apenas guias turísticos.
            * Tente variar os tipos de vídeos (vlogs, documentários curtos, entrevistas, etc.) para oferecer diferentes perspectivas.
            * **Para as buscas no Google, utilize termos que provavelmente retornem vídeos incorporáveis, como "vlog morando em [destino] youtube", "experiência de vida [destino] youtube", "como é viver em [destino] youtube".**

            **Formato de Saída Esperado:**

            Para [Nome do País/Cidade 1]:
            - Vídeo 1: [Título do Vídeo] - [Link para o YouTube] - [Breve Descrição]
            - Vídeo 2: [Título do Vídeo] - [Link para o YouTube] - [Breve Descrição]
            - Vídeo 3: [Título do Vídeo] - [Link para o YouTube] - [Breve Descrição]

            Para [Nome do País/Cidade 2]:
            - ... (seguir o mesmo formato)

            Para [Nome do País/Cidade 3]:
            - ... (seguir o mesmo formato)
            """
    )
    entrada_do_agente_cultural = f"Destinos: {destinos}\nPerfil do Usuário: {perfil}"
    insights = call_agent(insights_culturais, entrada_do_agente_cultural)
    return insights

In [232]:
######################################################################
# --- Agente 5: Agente de Geração de Documento --- #
######################################################################

# --- Função para geração de relatório HTML estilizado para exportação em PDF ---

def agente_gerador_html(perfil, destinos_info, burocracia_info, custo_vida_info, insights_culturais_info):
    agente_html = Agent(
        name="agente_gerador_html",
        model=MODEL_ID,
        description="Agente especializado em gerar um documento HTML completo e bonito, pronto para exportação em PDF.",
        tools=[],
        instruction= f"""
Você é um agente especialista em gerar relatórios profissionais em HTML para exportação em PDF.
Sua tarefa é organizar todas as informações recebidas em um documento HTML estilizado, pronto para ser convertido em PDF, seguindo as diretrizes:

**Diretrizes de estrutura:**
- Inclua um cabeçalho visual com o título do relatório.
- Após o título, traga uma breve introdução contextualizando o perfil do usuário e os destinos sugeridos.
- Para cada destino, crie uma seção destacada (card ou box), contendo:
    - Nome do país/cidade (título)
    - Subtítulos para cada categoria: Burocracia Migratória, Custo de Vida e Moradia, Insights Culturais e Vivenciais.
    - Sempre que possível, utilize **tabelas** para dados estruturados (custo, burocracia, etc.).
    - Links de vídeos devem estar destacados, com títulos e pequenas descrições.

**Diretrizes visuais (CSS):**
- Use uma fonte fácil de ler (ex: Segoe UI, Arial, sans-serif).
- Use cores sóbrias, com fundo branco ou cinza claro, e títulos com cor azul ou verde suave.
- Dê espaço entre seções e cards.
- Tabelas com bordas arredondadas e alternância de linhas (zebra).
- Use caixas (cards) com sombra leve para destacar cada destino.
- O relatório deve ser bonito e limpo, pensado para impressão em PDF.

**Exemplo de estrutura esperada:**
```html
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Relatório de Destinos para Migração</title>
  <style>
    body {{
      font-family: 'Segoe UI', Arial, sans-serif;
      background: #f8faff;
      color: #222;
      margin: 0;
      padding: 40px 0;
    }}
    .container  {{
      max-width: 900px;
      margin: auto;
      background: #fff;
      border-radius: 12px;
      box-shadow: 0 2px 16px rgba(0,0,0,0.08);
      padding: 36px 32px;
    }}
    h1  {{
      color: #1769aa;
      margin-bottom: 0.7em;
      font-size: 2.2em;
      border-bottom: 2px solid #eee;
      padding-bottom: 10px;
    }}
    h2  {{
      color: #29967d;
      margin-top: 2em;
      margin-bottom: 0.4em;
    }}
    h3  {{
      color: #222;
      margin-top: 1em;
      margin-bottom: 0.5em;
    }}
    .card  {{
      background: #f6fbff;
      border-radius: 10px;
      box-shadow: 0 1px 6px rgba(23,105,170,0.06);
      padding: 28px 24px;
      margin: 2.4em 0 2em 0;
    }}
    table  {{
      width: 95%;
      border-collapse: collapse;
      margin: 16px 0;
      font-size: 1.06em;
    }}
    th, td  {{
      padding: 10px 12px;
      border: 1px solid #c4e0ec;
      text-align: left;
    }}
    th  {{
      background: #e8f4fa;
      color: #1769aa;
      font-weight: 700;
    }}
    tr:nth-child(even)  {{
      background: #f2faff;
    }}
    .video-link  {{
      display: block;
      margin-bottom: 8px;
      color: #1569ca;
      text-decoration: none;
      font-weight: 600;
    }}
    .desc  {{
      color: #4e5d6c;
      margin-bottom: 14px;
      font-size: 0.97em;
    }}
    .footer  {{
      margin-top: 3em;
      color: #999;
      font-size: 0.98em;
      border-top: 1px solid #e5e5e5;
      padding-top: 18px;
      text-align: center;
    }}
  </style>
</head>
<body>
<div class="container">
  <h1>Relatório de Destinos para Migração</h1>
  <p>
    Com base no perfil informado ({perfil}), analisamos os destinos abaixo para você:
  </p>

  <div class="card">
    <h2>Canadá</h2>
    <h3>Burocracia Migratória</h3>
    <table>
      <tr><th>Documento</th><th>Exigido?</th><th>Observações</th></tr>
      <tr><td>Visto de Trabalho</td><td>Sim</td><td>Processo pode levar até 6 meses</td></tr>
      <tr><td>Permissão de Estudo</td><td>Opcional</td><td>Facilita residência futura</td></tr>
    </table>

    <h3>Custo de Vida e Moradia</h3>
    <table>
      <tr><th>Item</th><th>Valor Médio</th></tr>
      <tr><td>Aluguel</td><td>CAD$ 1.400/mês</td></tr>
      <tr><td>Supermercado</td><td>CAD$ 300/mês</td></tr>
    </table>

    <h3>Insights Culturais e Vivenciais</h3>
    <a class="video-link" href="https://youtu.be/exemplo1" target="_blank">Como é viver em Toronto?</a>
    <div class="desc">Vídeo com depoimento de imigrante brasileiro.</div>
    <a class="video-link" href="https://youtu.be/exemplo2" target="_blank">Mercado de Trabalho no Canadá</a>
    <div class="desc">Dicas para encontrar vagas e entender o ambiente profissional.</div>
  </div>

  <!-- Repita a estrutura .card para outros destinos -->

  <div class="footer">
    Este relatório foi gerado automaticamente por IA em 17/05/2025.
  </div>
</div>
</body>
</html>
```

**REGRAS IMPORTANTES:**
- Sempre siga a estrutura e estilo do exemplo acima.
- Nunca inclua instruções de uso ou markdown, apenas o HTML final.
- Substitua pelos dados reais de perfil, destinos, burocracia, custos e vídeos.
- Assegure clareza, boa divisão de seções e fácil leitura para exportação em PDF.

**Dado de entrada para o agente:**
Perfil do Usuário: {perfil}
Informações de Destino: {destinos_info}
Burocracia Migratória: {burocracia_info}
Custo de Vida e Moradia: {custo_vida_info}
Insights Culturais (Vídeos): {insights_culturais_info}
"""
    )
    entrada_html = f"""
    Perfil do Usuário: {perfil}
    Informações de Destino: {destinos_info}
    Burocracia Migratória: {burocracia_info}
    Custo de Vida e Moradia: {custo_vida_info}
    Insights Culturais (Vídeos): {insights_culturais_info}
    """
    documento_html = call_agent(agente_html, entrada_html)
    return documento_html


In [191]:
##################################################
# --- Agente 6: Agente de Resumo para Chat --- #
##################################################

def agente_resumo_chat(destinos_info, insights_culturais_info):
    agente_chat = Agent(
        name="agente_resumo_chat",
        model=MODEL_ID,
        description="Agente especializado em gerar um resumo conciso e amigável para apresentação em um chat, utilizando emojis.",
        tools=[],
        instruction= f"""
        Você é um agente especializado em criar resumos concisos e amigáveis para apresentação em um chat. Sua tarefa é pegar os dados coletados pelos outros agentes e gerar um resumo informativo para o usuário, formatado de forma clara e atraente, utilizando emojis para destacar informações importantes.

        **Informações Recebidas:**

        * **Informações de Destino:** "{destinos_info}"
        * **Insights Culturais e Vivenciais (Vídeos):** "{insights_culturais_info}"

        **Tarefa:**

        Gere um resumo para o chat, seguindo as seguintes diretrizes:

        1.  **Formato do Resumo:**
            * Apresente um resumo para cada destino.
            * Use emojis para tornar o resumo mais visual e atraente.
            * Use Markdown para formatar o texto (negrito, listas, etc.).
            * Estruture o resumo da seguinte forma:

        2.  **Conteúdo do Resumo:**
            * Para cada destino, inclua:
                * Nome do País/Cidade com um emoji de localização (e.g., 📍).
                * Um breve resumo (1-2 frases) dos principais pontos (por que o destino é adequado ao perfil, aspectos importantes da burocracia, visão geral do custo de vida e um destaque cultural). Use emojis relevantes para destacar esses pontos (e.g., 💼 para trabalho, 🏠 para moradia, 💰 para custo de vida, 🎉 para cultura).
                * Links para os vídeos do YouTube, formatados como links Markdown e precedidos por um emoji de vídeo (e.g., 🎥).
                * Use emojis para separar as seções de cada destino.

        **Restrições e Diretrizes:**

        * Use um tom amigável e conciso.
        * Use emojis de forma apropriada para destacar informações importantes e tornar o resumo mais visualmente atraente.
        * Use Markdown para formatar o texto.
        * Seja breve, mas forneça informações suficientes para despertar o interesse do usuário.
        * Não inclua o link para o documento completo do Google Docs neste resumo.

        **Formato de Saída Esperado (para o Chat):**
        ```
        Resumo das Opções de Destino:

        [Emoji de separação]

        📍 [Nome do País/Cidade 1]: [Emoji] [Resumo dos principais pontos, usando emojis relevantes].

        🎥 Vídeos:
        * [Título do Vídeo 1](Link do Vídeo 1) - [Breve Descrição]
        * [Título do Vídeo 2](Link do Vídeo 2) - [Breve Descrição]
        * ...

        [Emoji de separação]

        📍 [Nome do País/Cidade 2]: [Emoji] [Resumo dos principais pontos, usando emojis relevantes].

        🎥 Vídeos:
        * ...

        [Emoji de separação]

        📍 [Nome do País/Cidade 3]: [Emoji] [Resumo dos principais pontos, usando emojis relevantes].
        🎥 Vídeos:
        * ...

        [Emoji de separação]
        ```
        """
    )
    entrada_chat = f"""
    Informações de Destino: {destinos_info}
    Insights Culturais (Vídeos): {insights_culturais_info}
    """
    resumo_para_chat = call_agent(agente_chat, entrada_chat)
    return resumo_para_chat


In [226]:
# Instalar as bibliotecas necessárias


def gerar_pdf(html_text):
    # 1. Salve o HTML diretamente
    with open("temp.html", "w", encoding="utf-8") as f:
        f.write(html_text)

    # 2. Converta o HTML para PDF
    import pdfkit
    pdfkit.from_file("temp.html", "meu_roteiro(by Passaport).pdf")

    # 3. Baixe o PDF gerado
    from google.colab import files
    files.download("meu_roteiro(by Passaport).pdf")



In [221]:
######################################################
# --- Agente Principal (Chatbot de Planejamento) --- #
######################################################

def contato_inical():
    mensagem_inicial = """
    👋 Olá! Seja bem-vindo ao Passaport

      Sou o Passaport, seu agente principal para ajudar você a planejar sua mudança de país.

      Estou aqui para guiar você em cada etapa do processo, desde a coleta inicial de informações
      até a apresentação dos resultados de forma clara e útil.

      Como funciona?

      Preencha o formulário abaixo com suas informações e preferências.

      Aguarde enquanto processamos seus dados.

      Veja recomendações e orientações personalizadas para sua jornada internacional!

    ⚡ Dica: Responda ao máximo de campos possível para receber sugestões ainda mais alinhadas ao seu perfil!
    """
    return mensagem_inicial

In [230]:
# Acionar o Agente de Análise de Perfil e Destino
def iniciar_agentes(perfil_usuario):
    print('----------------------------------------------------------------------------------------------------')
    print("\n⏳Estou buscando os melhores destinos....")
    resultado_analise = agente_analista(perfil_usuario)
    print('----------------------------------------------------------------------------------------------------')

    print("\nOpções de Destino Sugeridas:")
    display(to_markdown(resultado_analise))
    print('----------------------------------------------------------------------------------------------------')

    # Perguntar ao usuário se deseja prosseguir
    perguntar_acao(perfil_usuario, resultado_analise)


In [229]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def perguntar_acao(perfil_usuario, resultado_analise):
    # Cria opções para o usuário escolher
    opcoes = ['sim', 'não', 'sair']
    escolha = widgets.RadioButtons(
        options=opcoes,
        description='Prosseguir?',
        disabled=False
    )
    botao = widgets.Button(description="Confirmar", button_style="success")
    saida = widgets.Output()

    def ao_clicar(b):
        with saida:
            clear_output()  # Limpa a saída anterior a cada clique
            resposta = escolha.value
            if resposta == "não":
                print("Desconsiderando os destinos sugeridos...")
                # Chama a função desejada
                iniciar_agentes(f"""{perfil_usuario} desconsiderar esses destinos: {resultado_analise}""")
            elif resposta == "sair":
                print("\nOk, obrigado por usar o assistente de planejamento de mudança de país!")
            else:
                destinos = resultado_analise  # Assumindo que a saída é uma string com os destinos
                # Acionar os outros agentes
                print("\n⏳Verificando Burocracias.....")
                resultado_burocracia = agente_burocracia(destinos)
                """print("\nInformações de Burocracia Migratória:")
                display(to_markdown(resultado_burocracia))"""

                print("\n⏳Verificando Custo de Vida e Moradias.....")
                resultado_custo_vida = agente_custo_vida_moradia(destinos)
                """print("\nInformações de Custo de Vida e Moradia:")
                display(to_markdown(resultado_custo_vida))"""

                print("\n⏳Verificando Insights Culturais.....")
                resultado_cultural = agente_insights_culturais(destinos, perfil_usuario)
                """print("\nInsights Culturais (Vídeos):")
                display(to_markdown(resultado_cultural))"""

                print("\n⏳Gerando Resultado.....")
                resultado_final = agente_resumo_chat(destinos, resultado_cultural)
                print("\nProntinho, segue seu planejamento:\n\n")
                display(to_markdown(resultado_final))

                print("\n⏳Agora vou preparar um arquivo com informações dos seus destinos.....")
                resultado_html = agente_gerador_html(perfil_usuario, destinos, resultado_burocracia, resultado_custo_vida, resultado_cultural)
                print("\n⏳Preparando Conteúdo.....")
                gerar_pdf(resultado_html)

                print("\nEspero que estas informações sejam úteis! Boa viagem!! ✈")

    botao.on_click(ao_clicar)
    display(escolha, botao, saida)


In [218]:
from ipywidgets import widgets
from IPython.display import display
import requests

def get_country_names():
    url = "https://restcountries.com/v3.1/all"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        country_names = sorted([country['name']['common'] for country in data])
        return country_names
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar a lista de países: {e}")
        return ['Erro ao carregar países']

def processar_formulario(nome, idade, interesses_culinarios, area_trabalho, estilo_vida, genero, idioma, pais_vive, outras_preferencias):

    print('----------------------------------------------------------------------------------------------------')
    print("\nDados recebidos do formulário:")
    print(f"Nome: {nome}")
    print(f"Idade: {idade}")
    print(f"Interesses Culinários: {interesses_culinarios}")
    print(f"Área de Trabalho/Profissão: {area_trabalho}")
    print(f"Estilo de Vida: {estilo_vida}")
    print(f"Gênero: {genero}")
    print(f"Idioma: {idioma}")
    print(f"País onde Vive: {pais_vive}")
    print(f"Outras Preferências/Necessidades: {outras_preferencias}")
    print('----------------------------------------------------------------------------------------------------')

    print("\nRecebi seus dados! Agora vou procurar seu destino\n")

    dados = f"""
          Nome: {nome}
          Idade: {idade}
          Interesses Culinários: {interesses_culinarios}
          Área de Trabalho/Profissão: {area_trabalho}
          Estilo de Vida: {estilo_vida}
          Gênero: {genero}
          Idioma: {idioma}
          País onde Vive: {pais_vive}
          Outras Preferências/Necessidades: {outras_preferencias}
          """

    iniciar_agentes(dados)

    return ""

def exibir_formulario():
    lista_paises = get_country_names()

    nome_input = widgets.Text(
        value='',
        placeholder='Digite seu nome',
        description='Nome:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    idade_input = widgets.IntText(
        value=0,
        description='Idade:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='200px')
    )
    interesses_culinarios_input = widgets.Textarea(
        value='',
        placeholder='Ex: culinária italiana, vegana, etc.',
        description='Interesses Culinários:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    area_trabalho_input = widgets.Text(
        value='',
        placeholder='Ex: Engenharia, Educação, etc.',
        description='Profissão:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    estilo_vida_dropdown = widgets.Dropdown(
        options=['Selecione', 'Pacato', 'Movimentado', 'Natureza', 'Cidade'],
        value='Selecione',
        description='Estilo de Vida:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    genero_dropdown = widgets.Dropdown(
        options=['Selecione', 'Masculino', 'Feminino', 'Não Binário', 'Prefiro não dizer', 'Outro'],
        value='Selecione',
        description='Gênero:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    idioma_input = widgets.Text(
        value='',
        placeholder='Ex: Português, Inglês',
        description='Idioma:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    pais_vive_combobox = widgets.Combobox(
        placeholder='Digite ou selecione um país',
        options=lista_paises,
        description='País onde Vive:',
        ensure_option=False,
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    outras_preferencias_input = widgets.Textarea(
        value='',
        placeholder='Descreva outras preferências ou necessidades',
        description='Outras Preferências:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    enviar_button = widgets.Button(
        description='Enviar',
        disabled=False,
        button_style='success',
        tooltip='Clique para enviar os dados',
        icon='check',
        layout=widgets.Layout(width='400px')
    )

    def on_enviar_button_clicked(b):
        nome = nome_input.value
        idade = idade_input.value
        interesses_culinarios = interesses_culinarios_input.value
        area_trabalho = area_trabalho_input.value
        estilo_vida = estilo_vida_dropdown.value
        genero = genero_dropdown.value
        idioma = idioma_input.value
        pais_vive = pais_vive_combobox.value
        outras_preferencias = outras_preferencias_input.value
        resultado = processar_formulario(
            nome,
            idade,
            interesses_culinarios,
            area_trabalho,
            estilo_vida,
            genero,
            idioma,
            pais_vive,
            outras_preferencias
        )
        print(f"\n{resultado}")

    enviar_button.on_click(on_enviar_button_clicked)

    formulario = widgets.VBox([
        nome_input,
        idade_input,
        interesses_culinarios_input,
        area_trabalho_input,
        estilo_vida_dropdown,
        genero_dropdown,
        idioma_input,
        pais_vive_combobox,
        outras_preferencias_input,
        enviar_button
    ])


    display(formulario)

In [219]:
# Função para iniciar a conversa com o agente principal
def iniciar_planejamento_migracao():
    contato = contato_inical()

    display(Markdown(contato))

    print("\n\n")
    # Para exibir o formulário, basta chamar:
    exibir_formulario()
    #iniciar_agentes(perfil_usuario)

In [233]:
iniciar_planejamento_migracao()

 
    👋 Olá! Seja bem-vindo ao Passaport

      Sou o Passaport, seu agente principal para ajudar você a planejar sua mudança de país.

      Estou aqui para guiar você em cada etapa do processo, desde a coleta inicial de informações 
      até a apresentação dos resultados de forma clara e útil.

      Como funciona?

      Preencha o formulário abaixo com suas informações e preferências.

      Aguarde enquanto processamos seus dados.

      Veja recomendações e orientações personalizadas para sua jornada internacional!

    ⚡ Dica: Responda ao máximo de campos possível para receber sugestões ainda mais alinhadas ao seu perfil!
    

----------------------------------------------------------------------------------------------------

Dados recebidos do formulário:
Nome: Filippe
Idade: 36
Interesses Culinários: Culinária Gaucha, Baina, Italiana
Área de Trabalho/Profissão: TI
Estilo de Vida: Pacato
Gênero: Masculino
Idioma: Inglês, Português 
País onde Vive: Brazil
Outras Preferências/Necessidades: Quero um lugar para viver com a minha família
----------------------------------------------------------------------------------------------------

Recebi seus dados! Agora vou procurar seu destino

----------------------------------------------------------------------------------------------------

⏳Estou buscando os melhores destinos....
----------------------------------------------------------------------------------------------------

Opções de Destino Sugeridas:


> Opção de Destino 1: Portugal - Portugal oferece um estilo de vida pacato, segurança para a família, um mercado de TI em crescimento e a facilidade do idioma português, além de influências culinárias próximas à italiana.
> 
> Opção de Destino 2: Canadá - O Canadá possui um mercado de trabalho forte em TI, é conhecido pela segurança e qualidade de vida para famílias, e tem uma comunidade brasileira significativa, o que pode facilitar a adaptação.
> 
> Opção de Destino 3: Itália - A Itália oferece uma rica cultura culinária, incluindo a italiana que é do interesse de Filippe, um estilo de vida mais tranquilo em algumas regiões e oportunidades no setor de TI, especialmente no norte do país.
> 
> Com base no perfil de Filippe e nas informações obtidas, aqui estão três opções de destino detalhadas, considerando seus interesses culinários, área de trabalho em TI, estilo de vida pacato e necessidade de um lugar para viver com a família:
> 
> **Opção de Destino 1: Portugal**
> 
> *   **Justificativa:** Portugal se destaca pela facilidade do idioma português, o que facilita a adaptação de Filippe e sua família. Além disso, o país oferece um custo de vida razoável em comparação com outros países da Europa Ocidental e um crescente mercado de TI, com muitas oportunidades em Lisboa e Porto. Portugal também é conhecido por sua segurança e qualidade de vida, tornando-o um local ideal para famílias. A culinária portuguesa tem influências mediterrâneas e atlânticas, com muitos pratos à base de peixe e frutos do mar, além de pratos de carne e enchidos que podem agradar aos seus interesses culinários.
> 
> *   **Considerações adicionais:**
>     *   **Culinária:** A culinária portuguesa é rica e diversificada, com influências mediterrâneas e atlânticas, oferecendo uma variedade de pratos à base de peixe, frutos do mar, carne e enchidos. A proximidade com a culinária italiana também pode ser um ponto positivo.
>     *   **TI:** O mercado de TI em Portugal está em crescimento, com oportunidades em áreas como desenvolvimento de software, cibersegurança e análise de dados. O Tech Visa facilita a imigração para talentos de TI de fora da União Europeia.
>     *   **Estilo de vida:** Portugal oferece um estilo de vida pacato e seguro, com muitas opções de lazer ao ar livre e uma rica cultura histórica. Cidades como Braga e Coimbra oferecem um custo de vida mais acessível e excelente qualidade de vida.
> 
> **Opção de Destino 2: Canadá**
> 
> *   **Justificativa:** O Canadá possui um dos mercados de trabalho mais fortes em TI, oferecendo inúmeras oportunidades para profissionais qualificados. Além disso, o país é conhecido pela segurança, alta qualidade de vida e excelentes serviços públicos, como saúde e educação, tornando-o um local ideal para criar uma família. O Canadá também possui uma grande comunidade brasileira, o que pode facilitar a adaptação de Filippe e sua família. Embora a culinária canadense não seja tão famosa quanto a italiana ou a gaúcha, as grandes cidades como Toronto, Montreal e Vancouver oferecem uma ampla variedade de restaurantes e mercados com culinárias de todo o mundo.
> 
> *   **Considerações adicionais:**
>     *   **Culinária:** Embora a culinária canadense não seja tão reconhecida, as grandes cidades oferecem uma variedade de restaurantes e mercados com culinárias de todo o mundo, incluindo brasileira e italiana.
>     *   **TI:** O mercado de trabalho em TI no Canadá está em alta, com muitas oportunidades em áreas como desenvolvimento de software, análise de dados e segurança da informação. Cidades como Toronto, Montreal e Vancouver são grandes centros de tecnologia.
>     *   **Estilo de vida:** O Canadá oferece um estilo de vida seguro e com alta qualidade de vida, com excelentes serviços públicos e muitas opções de lazer ao ar livre. A diversidade cultural e a presença de uma comunidade brasileira podem facilitar a adaptação.
> 
> **Opção de Destino 3: Itália**
> 
> *   **Justificativa:** A Itália é o berço da culinária italiana, um dos principais interesses de Filippe. Além disso, o país oferece um estilo de vida mais tranquilo em algumas regiões e oportunidades no setor de TI, especialmente no norte do país, perto de Milão. A Itália também é rica em história e cultura, oferecendo uma experiência de vida única. Para os brasileiros, a cidadania italiana pode ser uma grande vantagem, facilitando a entrada e permanência no país.
> 
> *   **Considerações adicionais:**
>     *   **Culinária:** A Itália oferece uma rica cultura culinária, com pratos famosos como pizza, massa, risotto e tiramisu. Cada região tem suas próprias especialidades, com influências diversas do norte ao sul do país.
>     *   **TI:** O mercado de TI na Itália está em crescimento, com oportunidades especialmente no norte do país. A validação de diplomas e o conhecimento do idioma italiano podem ser importantes para a inserção no mercado de trabalho.
>     *   **Estilo de vida:** A Itália oferece um estilo de vida mais tranquilo em algumas regiões, com cidades menores e paisagens rurais encantadoras. No entanto, o custo de vida pode ser alto em cidades como Milão e Roma.
> 


----------------------------------------------------------------------------------------------------


RadioButtons(description='Prosseguir?', options=('sim', 'não', 'sair'), value='sim')

Button(button_style='success', description='Confirmar', style=ButtonStyle())

Output()